# Practice Session 1: Policy and Value Iteration

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/rlsummerschool/practical-sessions/blob/master/notebooks/Practice_1.ipynb)

In [ ]:
# Set up the environment
# TODO: this will point to rlsummerschool's repo, once it's made bublic
!pip install "git+https://github.com/cipollone/rlss-practice1.git" --quiet

import time
import random

## RL development in Python

Large RL projects are usually developed locally, as any other Python package, in OOP, version controlled by Git (see [example](https://github.com/rlsummerschool/practical-sessions/blob/master/rlss_practice/environments.py)).

[Jupyter](https://jupyter.org/) notebooks, like this one, are collections of text cells and code cells, that can be executed with an interactive interpreter.
Colab notebooks are Jupyter notebooks whose interpreter runs in a runtime that is hosted by Google.

### Environments and tasks

An MDP is a model of Reinforcement Learning tasks. It comprises:
* a finite **state space** $\mathcal{S}$
* a finite **action space** $\mathcal{A}$
* a **starting-state distribution** $\nu_0(s)$
* a **reward function** $r: \mathcal{S}\times\mathcal{A}\rightarrow [0,1]$ (or vector $r\in[0,1]^{|\mathcal{S}|\cdot|\mathcal{A}|}$)
* a **transition function** $p: \mathcal{S}\times\mathcal{A}\rightarrow \Delta_{\mathcal{S}}$ (or matrix $p \in\mathbb{R}^{|\mathcal{S}|\cdot|\mathcal{A}|\times|\mathcal{S}|}$)


<div>
<img src="https://drive.google.com/uc?export=view&id=11KyhHxuaileBEJ9fFZmIS5-GzncQgyqt" alt="agent-environment interaction" width="600"/>
</div>


### The Gym(nasium) interface

[Gymnasium](https://gymnasium.farama.org/) is a standard API for Decision Processes, based on OpenAI's Gym library.

In [ ]:
# Standard import
import gymnasium as gym
from gymnasium import Env

The library also defines a number of classic benchmarks [Atari](https://gymnasium.farama.org/environments/atari/) games, [MuJoCo](https://gymnasium.farama.org/environments/mujoco/) simulations, and [ToyText](https://gymnasium.farama.org/environments/toy_text/), minimal environments for debugging.


In [ ]:
# A registered environment can be instantiated with
env = gym.make("CliffWalking-v0")

# Important properties:
print(env.action_space)
print(env.observation_space)

We only consider discrete state and action spaces. See the possible alternatives: `dir(gym.spaces)`

Environments can be fully observable, partially, or non-stationary. All these variants fit the same environment interface.
We should know which class our environment belongs to.
Today, we only consider stationary MDPs (observations are states).

The environment interfact allows  to sample initial states and transition with `env.reset()` and `env.step(action)`. Every method is well documented (see `help(gym.Env.step)`).

Try it out!

In [ ]:
observation, info = env.reset()
print(f"Initial observation: {observation}")

action = 0  # any action
observation, reward, terminated, truncated, info = env.step(action)

print(f"Observation: {observation}, reward {reward}")
observation, reward, terminated, truncated, info = env.step(action)
print(f"Observation: {observation}, reward {reward}")
observation, reward, terminated, truncated, info = env.step(action)
print(f"Observation: {observation}, reward {reward}")

Custom environments can be created by subclassing the `Env` class.

Environments can be also modified with wrappers. See the predefined Gymnasium [Wrappers](https://gymnasium.farama.org/api/wrappers/#gymnasium-wrappers).

## Setting up our RL task



We exeriment with a simple grid-world environment, based on the implementation in [minigrid](https://minigrid.farama.org/environments/minigrid/). The environment is configured and observations are transformed appropriately (you already know how).

In [ ]:
from rlss_practice.environments import Room, Rooms, MinigridBase

# Initializing the environment
env = Room(
    failure=0.0,
    agent_start_pos=(1, 1),
    agent_start_dir=0,
    size=5,
)
print(env)

In the ASCII representation, > is the agent, facing right, G is the goal, and W are walls.

Understand you own environment:

In [ ]:
print("Action space:", env.action_space)
print("Observation space:", env.observation_space, end="\n\n")
#print(help(MinigridBase))

    class MinigridBase
       MinigridBase(minigrid: minigrid.minigrid_env.MiniGridEnv, seed: int, failure=0.0)

       Base class for minigrid environments with explicit transition and reward functions.

       The agent is rewarded upon reaching the goal location.

       Action space:

       | Num | Name         | Action       |
       |-----|--------------|--------------|
       | 0   | left         | Turn left    |
       | 1   | right        | Turn right   |
       | 2   | forward      | Move forward |

       Observation space:

       | Name | Description             |
       |------|-------------------------|
       | x    | x coordinate            |
       | y    | y coordinate (downward) |
       | dir  | cardinal direction      |

       The transition function is stored in `T`,
       where `T[state][action][next_state]` is the transition probability.
       The reward function is `R`. `R[state][action]` contains a reward.

### Demo

Here's a rollout loop, for a single trajectory

In [ ]:
# Test it
done = False
observation, info = env.reset()
print("Initial observation:", observation)
print(env)

# Steps
while not done:

    # Action selection
    action = int(input("Action: "))

    # Transition
    observation, reward, terminated, truncated, info = env.step(action)
    done = terminated or truncated

    print(f"Observation: {observation}, reward {reward}")
    print(env)

We can define a function that performs `n_trajectories` rollouts on the environment with the given policy. We can also compute arbitrary statistics in the meanwhile.

In [ ]:
def rollouts(env, policy, n_trajectories, gamma):
    """Execute policy over env for n_trajectories and compute discounted return."""
    total_return = 0.0

    # Trajectores
    for _ in range(n_trajectories):

        # Init
        discount = 1.0
        ret = 0.0
        observation, info = env.reset()
        done = False

        # Steps
        while not done:

            # Action selection
            action = policy(observation)

            # Transition
            observation, reward, terminated, truncated, info = env.step(action)

            done = terminated or truncated
            ret += reward * discount
            discount *= gamma

            if done:
                total_return += ret

    env.close()
    return total_return / n_trajectories

Since we don't have a policy, yet. Let's define the uniform one.

In [ ]:
class UniformPolicy():
    def __init__(self, n_actions: int):
        self.n_actions = n_actions

    def __call__(self, observation):
        return random.randint(0, self.n_actions-1)

Let's try:

In [ ]:
avg_return = rollouts(
    env=env,
    policy=UniformPolicy(env.action_space.n),
    n_trajectories=20,
    gamma=0.9
)
print(avg_return)

We can also visualize the execution:

In [ ]:
from rlss_practice.wrappers import Renderer
visible_env = Renderer(env)

In [ ]:
avg_return = rollouts(
    env=visible_env,
    policy=UniformPolicy(env.action_space.n),
    n_trajectories=1,
    gamma=0.9
)
print(avg_return)

In [ ]:
visible_env.play()

### Explicit model

These algorithms assume that a complete model of the environment is available, in the form of explicit transition and reward functions. This is not part of the gym interface.

These two functions are stored in two members:

In [ ]:
# Transition and reward functions
T = env.T
R = env.R

# These are represented as dictionaries (for maximum clarity)
#   and indexed as T[state][action][next_state]
print("A few probabilities")
print(T[(1, 1, 0)][2][(2, 1, 0)])
print(T[(1, 1, 0)][2][(1, 1, 1)])
print(T[(3, 1, 0)][1][(3, 1, 1)])

print("\nA few rewards")
print(R[(1, 1, 0)][2])
print(R[(3, 3, 0)][0])

In [ ]:
# Explicit set of states and actions
print("States", env.states)
print("Actions", env.actions)
print("Number of states", len(env.states))
print("Number of actions", len(env.actions))

In [ ]:
# Some classic imports for the rest of the notebook
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output, display
import matplotlib as mpl
import math

mpl.style.use('seaborn-v0_8')

## Solving the Grid-world task

We would like to find a **stationary deterministic memoryless policy** or mapping from states to actions $\pi : \mathcal{S}\rightarrow \mathcal{A}$, that lets us arrive at the **goal state** sooner.

In other words, our objective to find $\pi$ which maximizes the **discounted return** from any starting state represented by:

\begin{align*}
    \rho(\pi) &= \mathbb{E}_{s_0\sim\nu_0, s_{t+1}\sim p(\cdot|s_t,\pi(s_t))}\left[\sum_{t=0}^\infty\gamma^t r(s_t,\pi(s_t))\right]\\
    &= \sum_{s}\nu_0(s) V^{\pi}(s)
\end{align*}

with the **discount factor** $\gamma\in[0,1)$.


&nbsp;

---

&nbsp;

Recall that the **state value function** for $s\in \mathcal{S}$ denoted

\begin{align*}
    V^\pi(s) &= \mathbb{E}_{s_{t+1}\sim p(\cdot|s_t,\pi(s_t))}\left[\sum_{t=0}^\infty\gamma^t r(s_t,\pi(s_t))\bigg|s_0=s\right]\\
    &=  \sum_{a}\pi(a|s)Q^{\pi}(s,a)
\end{align*}

represents the value of being in state $x$ and following policy $\pi$ while the **action-value function** for $x\in \mathcal{S}, a\in \mathcal{A}$ denoted

$$ Q^\pi(s,a) = r(s,a) + \gamma\sum_{s'\in\mathcal{S}}p(s'|s,a)V^{\pi}(s')$$

is the value of first taking action $a=\pi(s)$ in state $s$ then following policy $\pi$.

By standard notation, the **Bellman operator** of policy $\pi$ acting on functions $V : \mathcal{S}\rightarrow \mathbb{R}$ (or vectors $V\in\mathbb{R}^{|\mathcal{S}|}$) is given as:

$$(T^{\pi}V)(s) =  \sum_{a}\pi(a|s)\bigg[r(s,a) + \gamma\sum_{s'\in\mathcal{S}}p(s'|s,a)V(s')\bigg],\quad s\in\mathcal{S}$$


&nbsp;

---

&nbsp;

Ideally, we would like to find an **optimal policy** $\pi^*$ with:

$$ \pi^*(s) \in \underset{a\in\mathcal{A}}{\arg\max} \bigg\{r(s,a) + \gamma\sum_{s'\in\mathcal{S}}p(s'|s,a)V^{\pi^*}(s')\bigg\},\quad x\in\mathcal{S}$$

that maximizes our immediate reward and future return.

The Bellman operator of $\pi^*$ (a.k.a the **Bellman optimality operator**) acting on functions $V : \mathcal{S}\rightarrow \mathbb{R}$ (or vectors $V\in\mathbb{R}^{|\mathcal{S}|}$) is given as:

$$(T^{*}V)(s) =  \max_{a}\bigg\{r(s,a) + \gamma\sum_{s'\in\mathcal{S}}p(s'|s,a)V(s')\bigg\},\quad s\in\mathcal{S}$$


&nbsp;

---

&nbsp;

With the transition and reward function of the grid-world task available to us, we attempt to find an optimal policy via **Dynamic programming**. Precisely, we implement **Policy Iteration** and **Value iteration** methods introduced in the first lecture.

&nbsp;

### Policy Iteration (PI) Recap

**Idea**

Gradually advance to $\pi^{*}$ from an initial guess $\pi_0$ through a series of **policy evaluation** and **policy improvement** steps.

&nbsp;

**Policy Evaluation step**

Given a policy $\pi_k$, compute $V^{\pi_k}$ as

* $V^{\pi_k} = (I - \gamma\, p_{\pi_k})^{-1}r_{\pi_k}$

or estimate $V^{\pi_k}$ as $V_{k}$ with the following iterations

  * Initialize $V$ as $\mathbf{0}$ when $k = 0$ and $V_{k-1}$ otherwise, let $\pi = \pi_k$

    <div>
    <img src="https://drive.google.com/uc?export=view&id=1QaMg7a6HELjYycAnm6RE9vnzHD_fnaCn" alt="iterative policy evaluation" width="600"/>
    </div>

    Return $V_k = V$. <!--$(T^{\pi_{k}})^{n_{k}}V_{k-1}$-->

&nbsp;

**Policy Improvement step**

Obtain $\pi_{k+1}$ as the greedy policy w.r.t $V^{\pi_k}$ (or $V_{k}$). That is,
$$ \pi_{k+1}(s) \in \underset{a\in\mathcal{A}}{\arg\max} \bigg\{r(s,a) + \gamma\sum_{s'\in\mathcal{X}}p(s'|s,a)V^{\pi_{k}}(s')\bigg\},\quad s\in\mathcal{S}$$

&nbsp;

#### Putting everything together

Starting from an arbitrary stationary deterministic markovian policy $\pi_{0}$, for $k = 0,1,2,\cdots, K$ do:
* Compute $V^{\pi_k}$ or estimate $V^{\pi_k}$ with $V_{k}=(T^{\pi_{k}})^{n_{k}}V_{k-1}$
* Obtain $\pi_{k+1}$ as the greedy policy w.r.t $V^{\pi_k}$



&nbsp;

#### Theoretical guarantee

PI with "exact" policy evaluation steps finds an optimal policy after $K = \mathcal{\tilde{O}}((\mathcal{SA - S})/(1-\gamma))$ iterations.


&nbsp;

#### Notation

*   $r_{\pi}\in\mathbb{R}^{|\mathcal{S}|}$ so that for $s\in\mathcal{S}$, $r_{\pi}(s) = r(s,\pi(s))$.
*   $p_{\pi}\in\mathbb{R}^{|\mathcal{S}|\times|\mathcal{S}|}$ so that for $s,s'\in\mathcal{S}$, $p_{\pi}(s'|s) = p(s'|s,\pi(s))$.
*   $n_{k}$ is the number of loops required to compute $V_{k}$

In [ ]:
class PolicyIteration1:
  """
  Implements policy iteration with exact policy evaluation
  """
  def __init__(self, env: Env, discount_factor: float, initial_policy = None):
    # Store
    self.env = env
    self.states = self.env.states
    self.n_states = len(self.states)
    self.actions = self.env.actions
    self.n_actions = len(self.actions)
    self.T = self.env.T
    self.R = self.env.R
    self.gamma = discount_factor
    self.policy = initial_policy

    # Default policy
    if self.policy == None:
      np.random.seed(4)
      self.policy = {state: np.random.randint(0, self.n_actions-1) for state in self.states}

    self.V = np.zeros((self.n_states,1))
    self.policy_stable = False
    self.V_logs = []


  def evaluate_policy(self):
    """
    Given π_{k} compute V^{π_{k}} = (I - \gamma p_{π_{k}})^{-1}r_{π_{k}}.
    """
    # Value
    A = np.eye(self.n_states,self.n_states) - self.gamma*self.get_p_pi()
    b = self.get_r_pi()
    self.V = np.linalg.solve(A, b)

    # Log
    self.V_logs.append({state: self.V[i].item() for i, state in enumerate(self.states)})
    return


  def get_policy(self):
    """
    Compute the greedy policy:
      π_{k+1}(s) = argmax_{a\in A} Q^{\pi_{k}}(s,a)
    where
      Q^{\pi_{k}}(s,a) = R(s,a) + gamma*<P(.|s,a),V^{\pi_{k}}>
    is the state-action value.
    """
    self.policy_stable = True

    for state in self.states:
      max_Q_value = self.get_expected_update(state, self.policy[state])

      for action in self.actions:
        Q_value = self.get_expected_update(state, action)

        if action != self.policy[state] and max_Q_value < Q_value:
          self.policy[state] = action
          max_Q_value = Q_value
          self.policy_stable = False
    return


  def get_expected_update(self, state, action):
    """
    Compute Bellman update at a state-action pair.

    input:
    state
    action
    discount factor (gamma)
    state values (state_values)

    output:
    r(s,a) + gamma <P(.|s,a),v>
    """
    value  = self.R[state][action]

    for snext_index, snext in enumerate(self.states):
      value += self.gamma * self.T[state][action][snext] * self.V[snext_index]

    return value


  def get_p_pi(self):
    """
    Given π_{k}, compute p_{π_{k}}
    """
    p_pi = np.zeros((self.n_states,self.n_states))

    for s_index, s in enumerate(self.states):
      for snext_index, snext in enumerate(self.states):
        p_pi[s_index, snext_index] = self.T[s][self.policy[s]][snext]

    return p_pi


  def get_r_pi(self):
    """
    Given π_{k}, compute r_{π_{k}}
    """
    r_pi = np.zeros((self.n_states,1))

    for i, state in enumerate(self.states):
      r_pi[i][0] = self.R[state][self.policy[state]]

    return r_pi


  def as_policy(self):
    """Return the policy dictionary as a callable"""
    def callable_policy(observation):
        return self.policy[tuple(observation.tolist())]
    return callable_policy

In [ ]:
class PolicyIteration2(PolicyIteration1):
  """
  Implements policy iteration with iterative policy evaluation
  """
  def __init__(self,
              env: Env,
              discount_factor: float,
              theta: float,
              initial_policy = None):

    super().__init__(env, discount_factor, initial_policy)
    self.theta = theta
    self.V = {state: 0 for state in env.states}


  def evaluate_policy(self):
    """
    Starting from previous value estimate V_{k-1}
    estimate value of policy π_{k} by recursively applying
    the Bellman operator of the policy T^π_{k} to V_{k-1} until update is stable.
    """
    max_value_gap = np.inf

    while max_value_gap > self.theta:
      max_value_gap = 0

      for state in env.states:
        prev_statevalue = self.V[state]
        self.V[state] = self.get_expected_update(state, self.policy[state])
        max_value_gap = max(max_value_gap, np.abs(prev_statevalue - self.V[state]))

    self.V_logs.append(self.V.copy())


  def get_expected_update(self, state, action):
    """
    Compute Bellman update at a state-action pair.

    input:
    state
    action
    discount factor (gamma)
    state values (state_values)

    output:
    r(s,a) + gamma <P(.|s,a),v>
    """
    value  = env.R[state][action]

    for snext in env.states:
      value += self.gamma*env.T[state][action][snext] * self.V[snext]

    return value

### Value Iteration (VI) Recap

**Idea**

Gradually advance to $\pi^{*}$ with a single **iterative policy evaluation** step and **policy improvement**.

&nbsp;

**Policy Evaluation step**

No arbitrary starting policy needed. Estimate $V^{\pi^*}$ as follows:

  * For $k = 0,1,2,\cdots, K$

    * Set $V = \mathbf{0}$ when $k = 0$ and $V_{k-1}$ otherwise
      <div>
      <img src="https://drive.google.com/uc?export=view&id=1fsi3ZZgqZ-p061AxvSdeluWZTlwSjnGj" alt="iterative policy evaluation" width="400"/>
      </div>

      return $V$ as $V_{k}$.

&nbsp;

**Policy Improvement step**

Obtain $\hat{\pi}^{*}$ as the greedy policy w.r.t $V_{K}$. That is,

  $$ \hat{\pi}^{*}(s) \in \underset{a\in\mathcal{A}}{\arg\max} \bigg\{r(s,a) + \gamma\sum_{s'\in\mathcal{X}}p(s'|s,a)V_{K}(s')\bigg\},\quad s\in\mathcal{S}$$

&nbsp;

#### Putting everything together

* Estimate $V^{\pi^*}$ with $V_{K} = (T^*)^{K}\mathbf{0}$
* Obtain $\hat{\pi}^{*}$ as the greedy policy w.r.t $V_{K}$



&nbsp;

#### Theoretical guarantee

VI finds an **$\mathbf{\varepsilon}$-optimal policy** ($\pi^{\varepsilon}$) satisfying

$\qquad V^* - V^{\pi^{\varepsilon}}\leq \varepsilon\, e$

after $K = \mathcal{O}(\ln(2\gamma/\varepsilon(1-\gamma)^2)/(1-\gamma))$ iterations.


In [ ]:
class ValueIteration:
    """
    Implements value iteration
    """
    def __init__(self,
                discount_factor: float,
                epsilon: float,
                num_iterations = None,
    ):
      self.gamma = discount_factor
      self.K = num_iterations
      self.V_logs = []

      if self.K is None:
        self.K = math.ceil(np.log((2*self.gamma)/(epsilon*(1-self.gamma)**2))/(1-self.gamma))

      self.policy = {state: 0 for state in env.states}
      self.V = {state: 0 for state in env.states}


    def evaluate_policy(self):
      """
      Starting from an arbitrary value V_0(s) = 0 for s in S
      estimate the V^π* by recursively applying
      the Bellman optimality operator of the policy T^* to V_0 for K steps.
      """

      for k in range(self.K):
        for state in env.states:
          self.V[state] = max(map(lambda action: self.get_expected_update(state, action), env.actions))

      self.V_logs.append(self.V.copy())


    def get_policy(self):
      """
      Update policy to be the greedy policy:
        π(s) = argmax_{a\in A} Q(s,a)
      where
        Q(s,a) = R(s,a) + gamma*<P(.|s,a),v>
      is the state-action value.
      """
      for state in env.states:
        max_Q_value = self.get_expected_update(state, self.policy[state])
        for action in env.actions:
          Q_value = self.get_expected_update(state, action)
          if action != self.policy[state] and max_Q_value < Q_value:
            self.policy[state] = action
            max_Q_value = Q_value
      return


    def get_expected_update(self, state, action):
      """
      Compute Bellman update at a state-action pair.

      input:
      state
      action
      discount factor (gamma)
      state values (state_values)

      output:
      r(s,a) + gamma <P(.|s,a),v>
      """
      value  = env.R[state][action]

      for snext in env.states:
        value += self.gamma * env.T[state][action][snext] * self.V[snext]

      return value

    def as_policy(self):
      """Return the policy dictionary as a callable"""
      def callable_policy(observation):
          return self.policy[tuple(observation.tolist())]
      return callable_policy

### Run and test

In [ ]:
discount_factor = 0.9
epsilon = 0.01

# Run iteration with exact policy evaluation step
PI_planner1 = PolicyIteration1(env, discount_factor)

while not PI_planner1.policy_stable:
  PI_planner1.evaluate_policy()
  PI_planner1.get_policy()

Visualization

In [ ]:
def visualize(vlogs, state, grid_size):
  grid_size -= 2   # walls

  # Initial state values
  fig1 = plt.figure(figsize=(4, 3))
  ax1 = fig1.subplots()
  ax1.set_ylabel("value of (1,1,0)")
  ax1.set_xlabel("iterations")
  initial_state_values = [value[(1, 1, 0)] for value in PI_planner1.V_logs]
  ax1.plot(range(len(initial_state_values)), initial_state_values)

  # States values over time
  some_values_over_time = [
    {(x, y): values[(x, y, o)] for (x, y, o) in env.states if o == 0}  # o is a fixed agent orientation
    for values in vlogs
  ]
  values_over_time = [
    np.array([[values[(x+1, y+1)] for x in range(grid_size)] for y in range(grid_size)])
    for values in some_values_over_time
  ]
  vmids = [(values.max() + values.min()) / 2 for values in values_over_time]
  vmin = min([values.min() for values in values_over_time])
  vmax = max([values.max() for values in values_over_time])

  steps = len(values_over_time)
  fig2 = plt.figure(figsize=(4 * steps, 3))
  axs2 = fig2.subplots(1, steps)
  if isinstance(axs2, plt.Axes):
    axs2 = [axs2]
    multistep = False
  else:
    multistep = True

  for i, ax in enumerate(axs2):
    ax.set_title(f"values at step {i}" if multistep else "values")
    ax.imshow(values_over_time[i], cmap="Blues", vmin=vmin, vmax=vmax)
    ax.xaxis.set_visible(False)
    ax.yaxis.set_visible(False)
    for x in range(grid_size):
      for y in range(grid_size):
        val = values_over_time[i][y,x]
        ax.text(x, y, f"{val:.2f}", ha="center", va="center", color="w" if val > vmids[i] else "k")

In [ ]:
visualize(PI_planner1.V_logs, (1, 1, 0), 5)

We can also test the policy.

In [ ]:
avg_return = rollouts(
    env=visible_env,
    policy=PI_planner1.as_policy(),
    n_trajectories=20,
    gamma=0.9
)
print(avg_return)

In [ ]:
visible_env.play()

What if we have failure probabilities now?

In [ ]:
# Define
env = Room(
    failure=0.2,
    agent_start_pos=(1, 1),
    agent_start_dir=1,
    size=6,
)
visible_env = Renderer(env)
print(env)

# Plan
PI_planner1 = PolicyIteration1(env, discount_factor)
while not PI_planner1.policy_stable:
  PI_planner1.evaluate_policy()
  PI_planner1.get_policy()


In [ ]:
# Visualize
avg_return = rollouts(
    env=visible_env,
    policy=PI_planner1.as_policy(),
    n_trajectories=20,
    gamma=0.9
)
print(avg_return)

In [ ]:
visible_env.play()

visualize(PI_planner1.V_logs, (1, 1, 1), 6)

Now we run policy iteration with the approximate policy evaluation step

In [ ]:
theta = 0.1
PI_planner2 = PolicyIteration2(env, discount_factor, theta)

while not PI_planner2.policy_stable:
  PI_planner2.evaluate_policy()
  PI_planner2.get_policy()

In [ ]:
avg_return = rollouts(
    env=visible_env,
    policy=PI_planner2.as_policy(),
    n_trajectories=10,
    gamma=0.9
)
print(avg_return)

In [ ]:
visible_env.play()

visualize(PI_planner2.V_logs, (1, 1, 1), 6)

Finally, we also plan with Value Iteration.

In [ ]:
VI_planner = ValueIteration(discount_factor, epsilon)

VI_planner.evaluate_policy()
VI_planner.get_policy()
policy = VI_planner.as_policy()

In [ ]:
avg_return = rollouts(
    env=visible_env,
    policy=VI_planner.as_policy(),
    n_trajectories=20,
    gamma=0.9
)
print(avg_return)

In [ ]:
visible_env.play()

visualize(VI_planner.V_logs, (1, 1, 0), 6)

Credit:

*   Bruno Scherrer, "a lecture on Markov Decision Processes and Dynamic Programming", June 2023, [link text](https://)
*   Csaba Szepesvári "a lecture series on Theoretical Foundations of Reinforcement Learning", 2020, [RL Theory course](https://rltheory.github.io/)
*   Richard S. Sutton, Andrew G. Barto "Reinforcement Learning: An Introduction", second edition, 2020, [Book](http://incompleteideas.net/book/RLbook2020.pdf)



